In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Fivestars Store by City
Lists top cities in Fivestars from the perspective of store count number.

# Store Count by Business Category

In [2]:
import MySQLdb
import os
host = os.environ["FS_MYSQL_HOST"]
user = os.environ["FS_MYSQL_USER"]
db = os.environ["FS_MYSQL_DB"]
passwd = os.environ["FS_MYSQL_PASSWD"]

conn = MySQLdb.connect(host=host, user=user, db=db, passwd=passwd)
cursor = conn.cursor()

import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('FangwenYu', 'in8lq23u70')

def format_ds(ds):
    return '{}/{}/{}'.format(ds[:4], ds[4:6], ds[6:])

cursor.execute("select category from report_fs_store_by_category_stats where ds='20160219' order by category")
categories = cursor.fetchall()
data = {}
data['Business Category'] = [value[0] for value in categories]
columns = ['Business Category']
cursor.execute('select distinct ds from report_fs_store_by_category_stats order by ds')
ds_list = cursor.fetchall()


report_date = []
for ds in ds_list:
    curr_ds = ds[0]
    report_date.append(format_ds(curr_ds))
    sql = "select store_cnt from report_fs_store_by_category_stats where ds='{}' order by category".format(curr_ds)
    cursor.execute(sql)
    values= cursor.fetchall()
    data[curr_ds] = [value[0] for value in values]   
    columns.append(curr_ds)

import pandas as pd
df = pd.DataFrame(data, columns=columns)
df.head(50)


,Business Category,20160219,20160226,20160304,20160311
0,Arts & Entertainment,166,170,165,164
1,Automotive,110,111,109,117
2,Bakeries,135,134,133,138
3,Beauty & Spas,551,567,565,567
4,Coffee & Tea,493,495,498,509
5,Desserts,350,351,348,352
6,Diners,943,961,955,961
7,Fitness & Instruction,216,215,218,220
8,Formal Restaurants,1517,1525,1529,1527
9,General Retails,2204,2209,2230,2242


In [48]:
df2 = df.transpose()
data = []
for category in df2.columns:
    trace = go.Scatter(
        x = report_date,
        y = df2[category].tolist()[1:],
        name = df2[category].tolist()[0],
        mode = 'lines+markers'
    )
    data.append(trace)

    
layout = dict(title = 'Store Count by Business Category',
              xaxis = dict(title = "Report Date"),
              yaxis = dict(title = 'Store Count'),
             )

fig = dict(data=data, layout=layout)
py.iplot(data, filename='FS Store Count by Business Category Timeline')

# import plotly
# plotly.offline.plot(fig, filename='test.html')


## Store New Count Timeline

In [56]:
cursor.execute("select distinct ds from report_fs_store_change_stats where op_flag='New' and ds > '20160219' order by ds")
ds_list = cursor.fetchall()

cursor.execute("select distinct yelp_sub_category from stage_fs_yelp_category_mapping order by yelp_sub_category")
categories = cursor.fetchall()
data = {}
data['Business Category'] = [value[0] for value in categories]
columns = ['Business Category']

for ds in ds_list:
    curr_ds = ds[0]
    sql = """
select a.yelp_sub_category, case when b.store_cnt is null then 0 else b.store_cnt end as store_cnt
from 
(
select distinct yelp_sub_category from stage_fs_yelp_category_mapping
) a 
left join 
(
select yelp_sub_category, count(*) as store_cnt 
from report_fs_store_change_stats 
where op_flag = 'New'
and   ds = '{}'
group by ds, yelp_sub_category
) b
on a.yelp_sub_category = b.yelp_sub_category
order by a.yelp_sub_category
""".format(curr_ds)
    
    cursor.execute(sql)
    values= cursor.fetchall()
    data[curr_ds] = [value[1] for value in values]   
    columns.append(curr_ds)

import pandas as pd
df = pd.DataFrame(data, columns=columns)
df.head(50)

,Business Category,20160226,20160304,20160311
0,Arts & Entertainment,4,1,1
1,Automotive,4,4,14
2,Bakeries,1,3,8
3,Beauty & Spas,24,14,11
4,Coffee & Tea,8,6,15
5,Desserts,5,3,7
6,Diners,26,9,19
7,Fitness & Instruction,2,7,3
8,Formal Restaurants,29,24,24
9,General Retails,31,54,42


## Store Lose Count Timeline

In [57]:
cursor.execute("select distinct ds from report_fs_store_change_stats where op_flag='Lose' and ds > '20160219' order by ds")
ds_list = cursor.fetchall()

cursor.execute("select distinct yelp_sub_category from stage_fs_yelp_category_mapping order by yelp_sub_category")
categories = cursor.fetchall()
data = {}
data['Business Category'] = [value[0] for value in categories]
columns = ['Business Category']

for ds in ds_list:
    curr_ds = ds[0]
    sql = """
select a.yelp_sub_category, case when b.store_cnt is null then 0 else b.store_cnt end as store_cnt
from 
(
select distinct yelp_sub_category from stage_fs_yelp_category_mapping
) a 
left join 
(
select yelp_sub_category, count(*) as store_cnt 
from report_fs_store_change_stats 
where op_flag = 'Lose'
and   ds = '{}'
group by ds, yelp_sub_category
) b
on a.yelp_sub_category = b.yelp_sub_category
order by a.yelp_sub_category
""".format(curr_ds)
    
    cursor.execute(sql)
    values= cursor.fetchall()
    data[curr_ds] = [value[1] for value in values]   
    columns.append(curr_ds)

import pandas as pd
df = pd.DataFrame(data, columns=columns)
df.head(50)

,Business Category,20160226,20160304,20160311
0,Arts & Entertainment,0,6,2
1,Automotive,3,6,6
2,Bakeries,2,4,3
3,Beauty & Spas,8,16,9
4,Coffee & Tea,6,3,4
5,Desserts,4,6,3
6,Diners,8,15,13
7,Fitness & Instruction,3,4,1
8,Formal Restaurants,21,20,26
9,General Retails,26,33,30


## Top 20 Cities that have most stores

In [20]:
cursor.execute("""select ds as "Report Date", country as "Country", state as "State", city as "City", store_cnt as "Store Count" from 
report_fs_store_by_city_stats where category = 'Top20_cnt' order by ds desc, store_cnt desc limit 100""")
cities = cursor.fetchall()
data = [[item for item in value] for value in cities]
columns = ['Report Date', 'Country', 'State', 'City', 'Store Count']

import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df = pd.DataFrame(data, columns=columns)
df.head(100)


,Report Date,Country,State,City,Store Count
0,20160311,CA,ON,Toronto,204
1,20160311,US,IL,Chicago,134
2,20160311,US,TX,Austin,127
3,20160311,US,CA,San Francisco,121
4,20160311,US,CA,San Diego,115
5,20160311,US,CA,Los Angeles,109
6,20160311,US,TX,Houston,97
7,20160311,US,FL,Miami,92
8,20160311,US,OR,Portland,89
9,20160311,US,CA,San Jose,88


## Top 20 Cities that have new stores

In [6]:
cursor.execute("""select ds as "Report Date", country as "Country", state as "State", city as "City", store_cnt as "Store Count" from 
report_fs_store_by_city_stats where category = 'Top20_New' order by ds desc, store_cnt desc limit 100""")
cities = cursor.fetchall()
data = [[item for item in value] for value in cities]
columns = ['Report Date', 'Country', 'State', 'City', 'Store Count']

df = pd.DataFrame(data, columns=columns)
df.head(100)

,Report Date,Country,State,City,Store Count
0,20160311,US,FL,Pembroke Pines,6
1,20160311,US,NY,New York,5
2,20160311,US,TX,Houston,4
3,20160311,US,TX,Austin,4
4,20160311,US,CO,Aurora,3
5,20160311,US,CA,San Gabriel,3
6,20160311,US,CA,Chico,2
7,20160311,CA,ON,Kitchener,2
8,20160311,US,TX,New Braunfels,2
9,20160311,US,FL,Miami,2


## Top 20 Cities that have lost stores

In [7]:
cursor.execute("""select ds as "Report Date", country as "Country", state as "State", city as "City", store_cnt as "Store Count" from 
report_fs_store_by_city_stats where category = 'Top20_Lose' order by ds desc, store_cnt desc limit 100""")
cities = cursor.fetchall()
data = [[item for item in value] for value in cities]
columns = ['Report Date', 'Country', 'State', 'City', 'Store Count']

df = pd.DataFrame(data, columns=columns)
df.head(100)

,Report Date,Country,State,City,Store Count
0,20160311,US,CA,San Diego,5
1,20160311,CA,SK,Saskatoon,4
2,20160311,US,IL,Chicago,4
3,20160311,CA,ON,Toronto,3
4,20160311,US,CA,Murrieta,2
5,20160311,US,CA,Santa Cruz,2
6,20160311,CA,ON,Ottawa,2
7,20160311,CA,ON,Guelph,2
8,20160311,CA,NL,St. John's,1
9,20160311,CA,ON,"Richmond Hill,",1
